In [295]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing  import LabelEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,FunctionTransformer
import warnings
warnings.filterwarnings('ignore')

In [296]:
df= pd.read_csv('ElecDeviceRatingPrediction_Milestone2.csv')
electro = df.copy()

In [297]:
trainElectro, testElectro = train_test_split(electro, test_size=0.2, random_state=0)

In [298]:
def convertingDtypes(df):
    df['ram_gb']=df['ram_gb'].str.strip('GB')
    df['Number of Ratings']=df['Number of Ratings'].replace(0, np.nan)
    df['reviews per rating']= df['Number of Reviews']/df['Number of Ratings']
    df['reviews - rating']= abs(df['Number of Reviews']-df['Number of Ratings'])
    df['reviews + rating']= df['Number of Reviews']+df['Number of Ratings']
    df['reviews * rating']= df['Number of Reviews']*df['Number of Ratings']
    #convert to the correct datatype
    df['Touchscreen']=df['Touchscreen'].astype('category')
    df['msoffice']=df['msoffice'].astype('category')
    df['rating']=df['rating'].astype('category')
    df['ram_gb']=df['ram_gb'].astype('int')
    df['ssd']=df['ssd'].str.strip('GB')
    df['ssd']=df['ssd'].astype('int')
    df['hdd']=df['hdd'].str.strip('GB')
    df['hdd']=df['hdd'].astype('int')
    df['processor_brand']=df['processor_brand'].astype('category')
    df['brand']=df['brand'].astype('category')
    df['processor_name']=df['processor_name'].astype('category')
    df['ram_type']=df['ram_type'].astype('category')
    df['os']=df['os'].astype('category')
    df['weight']=df['weight'].astype('category')
    df['graphic_card_gb']=df['graphic_card_gb'].str.strip('GB')
    df['graphic_card_gb']=df['graphic_card_gb'].astype('int')
    df['warranty']=df['warranty'].str.strip('year || years')
    df['warranty']=df['warranty'].str.replace('No warrant','0')
    df['warranty']=df['warranty'].astype('int')
    df['processor_gnrtn']=df['processor_gnrtn'].replace('Not Available',pd.NA)
    df['processor_gnrtn']=df['processor_gnrtn'].str.strip('th')
    df.drop_duplicates(inplace=True)
    return df

In [300]:
mode_value = trainElectro['processor_gnrtn'].mode()[0]

In [301]:
def handleOutliers(df):
    #drop the outliers
    df['processor_gnrtn']=df['processor_gnrtn'].astype('int')
    for col in df.columns:
            if (((df[col].dtype)=='float') | ((df[col].dtype)=='int32')|((df[col].dtype)=='int64')):
                percentiles = df[col].quantile([0.25,0.75]).values
                iqr= percentiles[1]-percentiles[0]
                df[col][df[col] <= percentiles[0]-1.5*iqr] = percentiles[0]
                df[col][df[col] >= percentiles[1]+1.5*iqr] = percentiles[1]
            else:
                df[col]=df[col]
    return df


In [302]:
from sklearn import metrics
import time
graphAcc={}
graphTime={}
electro=convertingDtypes(electro)
def testingModels(df,classifier):
    mode_value = df['processor_gnrtn'].mode()[0]
    df['processor_gnrtn'].fillna(mode_value,inplace=True)
    df['Number of Ratings'].fillna(df['Number of Ratings'].mean(),inplace=True)
    df['reviews per rating'].fillna(df['Number of Ratings'].mean(),inplace=True)
    df['reviews - rating'].fillna(df['reviews - rating'].mean(),inplace=True)
    df['reviews + rating'].fillna(df['reviews + rating'].mean(),inplace=True)
    df['reviews * rating'].fillna(df['reviews * rating'].mean(),inplace=True)
    df.drop_duplicates(inplace=True)
    label_encoder = LabelEncoder()  
    df['weight']= label_encoder.fit_transform(df['weight'])
    df['msoffice'] = label_encoder.fit_transform(df['msoffice'])
    df['ram_type'] = label_encoder.fit_transform(df['ram_type'])
    df['rating'] = label_encoder.fit_transform(df['rating'])
    df['Touchscreen'] = label_encoder.fit_transform(df['Touchscreen'])
    df=pd.get_dummies(df,columns=['brand','processor_brand','processor_name','os'],dtype=int)
    df=handleOutliers(df)
    #split to features and result
    X=df.drop(['rating'],axis=1)
    X_cat = df.drop(columns=['warranty' ,'Number of Ratings','Number of Reviews', 'Price','ram_gb','ssd','hdd','graphic_card_gb','reviews per rating','reviews - rating','reviews + rating','reviews * rating','rating'])
    X_numeric=df[['warranty' ,'Number of Ratings','Number of Reviews', 'Price','ram_gb','ssd','hdd','graphic_card_gb','reviews per rating','reviews - rating','reviews + rating','reviews * rating']]
    Y = df['rating']
    scaler =  MinMaxScaler(feature_range=(0,1))
    X= pd.DataFrame(scaler.fit_transform(X),index=X.index,columns= X.columns)
    best_features1 = SelectKBest(score_func=chi2, k=3)
    fit = best_features1.fit(X_cat,Y)

    # Summarize scores
    chi_scores = pd.DataFrame(fit.scores_, columns=["Chi-Square Score"])
    columns = pd.DataFrame(X_cat.columns, columns=["Feature"])
    chi_summary = pd.concat([columns, chi_scores], axis=1)
    top_chi_features = chi_summary.sort_values('Chi-Square Score', ascending=False).head(3)

    best_features2 = SelectKBest(score_func=f_classif, k=3)
    fit2 = best_features2.fit(X_numeric,Y)

    # Summarize scores
    ANOVA_scores = pd.DataFrame(fit2.scores_, columns=["Chi-Square Score"])
    columns2 = pd.DataFrame(X_numeric.columns, columns=["Feature"])
    ANOVA_summary = pd.concat([columns2, ANOVA_scores], axis=1)
    top_ANOVA_features = ANOVA_summary.sort_values('Chi-Square Score', ascending=False).head(3)
    lst=[]
    for i in top_ANOVA_features['Feature'].values:
        lst.append(i)
    for i in top_chi_features['Feature'].values:
        lst.append(i)
    X_new = X[lst]
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.2, random_state= 104, shuffle = True)
    model=classifier
    start =time.time()
    model.fit(X_train, y_train)
    end=time.time()
    trainingTime= end-start
    y_pred = model.predict(X_test) 
    y_pred_train = model.predict(X_train)
    testacc = metrics.accuracy_score(y_test, y_pred)
    trainacc= metrics.accuracy_score(y_train, y_pred_train)
    print("Accuracy test: ",testacc)
    print("Accuracy train: ",trainacc)
    graphAcc[str(classifier)]=testacc
    graphTime[str(classifier)]=trainingTime
    

In [303]:
#result=testingModels(electro,LogisticRegression(penalty='l2',max_iter=200))


In [304]:
# testingModels(electro,LogisticRegression(penalty='l1',max_iter=600,solver='saga'))

In [305]:
# testingModels(electro,LogisticRegression(penalty='elasticnet',max_iter=200,solver='saga',l1_ratio=0.5))

In [306]:
# from sklearn import svm
# testingModels(electro,svm.SVC(kernel='rbf',gamma=0.9,C=3))

In [307]:
# testingModels(electro,svm.SVC(kernel='linear',C=2))

In [308]:
# testingModels(electro,svm.SVC(kernel='poly',degree=6))


In [309]:
# from sklearn.tree import DecisionTreeClassifier
# testingModels(electro,DecisionTreeClassifier(max_depth=3,splitter='best'))

In [310]:
# testingModels(electro,DecisionTreeClassifier(max_depth=3,splitter='random'))

In [311]:
# testingModels(electro,DecisionTreeClassifier(max_depth=4,splitter='best'))

In [312]:
# graphAcc

In [313]:
# plt.bar(list(graphAcc.keys()) , list(graphAcc.values()))
# plt.xticks(rotation = 90)
# plt.show()

In [314]:
# plt.bar(list(graphTime.keys()) , list(graphTime.values()))
# plt.xticks(rotation = 90)
# plt.show()

In [315]:

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
column_trans = ColumnTransformer(
[('customConverting', FunctionTransformer(func=convertingDtypes), []),('imp_col1', SimpleImputer(strategy='constant',fill_value=trainElectro['processor_gnrtn'].mode()[0]), ['processor_gnrtn']),
  ('imp_col2', SimpleImputer(strategy='mean'), ['Number of Ratings','reviews per rating','reviews - rating','reviews + rating','reviews * rating']),
  ('customOutliers',FunctionTransformer(func=handleOutliers), []),
  ('encoder1',OneHotEncoder(),['brand','processor_brand','processor_name','os']),
  ('encoder2',LabelEncoder(),['ram_type','msoffice','weight','Touchscreen']),
  ('scaler', MinMaxScaler(),[]),
  ('numricalFeatures',SelectKBest(score_func=f_classif, k=3),['warranty' ,'Number of Ratings','Number of Reviews', 'Price','ram_gb','ssd','hdd','graphic_card_gb','reviews per rating','reviews - rating','reviews + rating','reviews * rating']),
  ('catFeatures', SelectKBest(score_func=chi2, k=3),'passthrough')],
remainder='passthrough')
# numeric=electro['warranty' ,'Number of Ratings','Number of Reviews', 'Price','ram_gb','ssd','hdd','graphic_card_gb','reviews per rating','reviews - rating','reviews + rating','reviews * rating']
pipeline = Pipeline([
    ('preprocessor', column_trans),
    ('classifier', LogisticRegression(penalty='l2',max_iter=200))  # Example classifier
])

In [317]:
import pickle
X_train = trainElectro.drop(['rating'],axis=1)
Y_train =trainElectro['rating']
pipeline.fit(X_train,Y_train)
with open('logistic.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

ValueError: A given column is not a column of the dataframe

In [ ]:
with open('model_pipeline.pkl', 'rb') as f:
    loaded_pipeline = pickle.load(f)
preprocessed_data = loaded_pipeline.transform(testElectro)
predictions = loaded_pipeline.predict(preprocessed_data)
# Calculate accuracy score
accuracy = accuracy_score(testElectro['rating'], predictions)
print("Accuracy:", accuracy)